In [1]:
import src.utilities.utils as utils 
test_data = utils.read_jsonl('data/snli_1.0/snli_1.0_test.jsonl')
train_data = utils.read_jsonl('data/snli_1.0/snli_1.0_train.jsonl')
dev_data = utils.read_jsonl('data/snli_1.0/snli_1.0_dev.jsonl')

In [2]:
# nums = ['2','3','4','5','6','7','8','9','10']
# special_numericals = ['some', 'many', 'all'] # TODO
# number_words = utils.plural_words_for_numbers + nums

convert_dict_plural = {
    'two':2,
    'three':3,
    'four':4,
    'five':5,
    'six':6,
    'seven':7,
    'eight':8,
    'nine':9,
    'ten':10,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10
}

def numerical_word_in_sentence(sentence):
    return [x for x in sentence.split() if x in convert_dict_plural]

def subset_with_numericals(dataset):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if sent1_num_words or sent2_num_words:
#             print((sent1_num_words, sent2_num_words))
#             print(data['gold_label'])
#             print(data['sentence1'])
#             print(data['sentence2'])
#             print()
            result.append(data)
    return result

def subset_with_same_numericals(dataset):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if sent1_num_words == sent2_num_words and len(sent1_num_words) == 1: # TODO: When len(list) > 1
#             print((sent1_num_words, sent2_num_words))
#             print(data['gold_label'])
#             print(data['sentence1'])
#             print(data['sentence2'])
#             print()
            result.append(data)
    return result

In [3]:
def subset_by_relationship_type(dataset, relation):
    result = []
    return [data for data in dataset if data['gold_label'] == relation]

In [4]:
test_data_entailment = subset_by_relationship_type(test_data, 'entailment')
train_data_entailment = subset_by_relationship_type(train_data, 'entailment')
dev_data_entailment = subset_by_relationship_type(dev_data, 'entailment')
# test_data_contradiction = subset_by_relationship_type(test_data, 'contradiction')
# test_data_neutral = subset_by_relationship_type(test_data, 'neutral')

In [5]:
len(test_data_entailment)

3368

In [6]:
len(train_data_entailment)

183416

In [7]:
len(dev_data_entailment)

3329

In [8]:
# Entailment pairs that include numerical words in either one of the sentences
test_entailment_with_numericals = subset_with_numericals(test_data_entailment)
train_entailment_with_numericals = subset_with_numericals(train_data_entailment)
dev_entailment_with_numericals = subset_with_numericals(dev_data_entailment)

In [9]:
# test_entailment_with_numericals

In [10]:
# Entailment pairs that include equal numerical words
test_entailment_with_same_numericals = subset_with_same_numericals(test_entailment_with_numericals)
train_entailment_with_same_numericals = subset_with_same_numericals(train_entailment_with_numericals)
dev_entailment_with_same_numericals = subset_with_same_numericals(dev_entailment_with_numericals)

In [21]:
# test_entailment_with_same_numericals

In [12]:
def get_binary_pairs(list_of_dic):
    res = []
    for each in list_of_dic:
        res.append(
            (each['sentence1_binary_parse'], each['sentence2_binary_parse'])
        )
    return res

In [13]:
test_binary_entailment_same_example = get_binary_pairs(test_entailment_with_same_numericals)
train_binary_entailment_same_example = get_binary_pairs(train_entailment_with_same_numericals)
dev_binary_entailment_same_example = get_binary_pairs(dev_entailment_with_same_numericals)

test_binary_entailment_same_example

[('( ( ( A man ) ( ( standing ( in ( front ( of ( ( a building ) ( on ( ( the phone ) ( as ( two men ) ) ) ) ) ) ) ) ) ( to ( ( the ( side pain ) ) ( on ( the side ) ) ) ) ) ) . )',
  '( ( ( a guy ) ( near ( a building ) ) ) ( stands ( by ( two ( other men ) ) ) ) )'),
 ('( ( ( ( A crowd ) ( of people ) ) ( ( looking up ) ( at ( ( 3 people ) ( on ( ( the edge ) ( of ( ( the roof ) ( of ( a building ) ) ) ) ) ) ) ) ) ) . )',
  "( ( ( The crowd ) ( on ( the ground ) ) ) ( ( is ( watching ( ( 3 people ) ( on ( ( the ( roof 's ) ) edge ) ) ) ) ) . ) )"),
 ('( ( ( Six or ) ( seven people ) ) ( ( are ( standing ( on ( ( ( ( a pier ) ( with ( a table ) ) ) and ) ( ( a pair ) ( of ( glasses ( in ( the foreground ) ) ) ) ) ) ) ) ) . ) )',
  '( ( ( Six or ) ( seven people ) ) ( ( are ( standing ( on ( a pier ) ) ) ) . ) )'),
 ('( ( A boy ) ( ( is ( ( riding down ) ( ( the road ) ( between ( two cows ) ) ) ) ) . ) )',
  '( ( ( ( A boy ) , ) ( ( ( a road ) and ) ( two cows ) ) ) . )'),
 ('( ( The 

In [14]:
numerical_word_in_sentence(test_binary_entailment_same_example[0][0])[0]

'two'

In [15]:
def generate_same_to_same_plural(binary_list, label):
    res = []
    for i in range(len(binary_list)):
        new_bin_pairs = utils.same_to_same_plural_number(
            binary_list[i],
            numerical_word_in_sentence(binary_list[i][0])[0]
        )
        for pair in new_bin_pairs:
            rem_bin_pair = utils.remove_parenthesis(pair)
            tmp_dict = dict()
            tmp_dict['gold_label'] = label
            tmp_dict['sentence1'] = rem_bin_pair[0]
            tmp_dict['sentence2'] = rem_bin_pair[1]
            tmp_dict['sentence1_binary_parse'] = pair[0]
            tmp_dict['sentence2_binary_parse'] = pair[1]
            res.append(tmp_dict)
    return res

In [16]:
test_entailment_same_to_same_plural = generate_same_to_same_plural(test_binary_entailment_same_example, 'entailment')
train_entailment_same_to_same_plural = generate_same_to_same_plural(train_binary_entailment_same_example, 'entailment')
dev_entailment_same_to_same_plural = generate_same_to_same_plural(dev_binary_entailment_same_example, 'entailment')


In [17]:
len(test_entailment_same_to_same_plural)

896

In [18]:
len(train_entailment_same_to_same_plural)

48480

In [19]:
len(dev_entailment_same_to_same_plural)

1056

In [20]:
import json
with open('test_entailment_same_to_same_plural.json', 'w') as outfile:
    json.dump(test_entailment_same_to_same_plural, outfile)

with open('train_entailment_same_to_same_plural.json', 'w') as outfile:
    json.dump(train_entailment_same_to_same_plural, outfile)
    
with open('dev_entailment_same_to_same_plural.json', 'w') as outfile:
    json.dump(dev_entailment_same_to_same_plural, outfile)